In [1]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_transactions = pd.read_csv("/content/drive/MyDrive/ITMO_ML/Data/transactions.csv", nrows=1000000, sep=",")
df_tr_types = pd.read_csv("/content/drive/MyDrive/ITMO_ML/Data/tr_types.csv", sep=";")
df_tr_mcc_codes = pd.read_csv("/content/drive/MyDrive/ITMO_ML/Data/tr_mcc_codes.csv", sep=";")
df_gender_train = pd.read_csv("/content/drive/MyDrive/ITMO_ML/Data/gender_train.csv", sep=",")

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [4]:
count_tr_types = 1000
tr_type_column_sample = df_transactions["tr_type"].sample(count_tr_types)

substring_1 = df_tr_types["tr_description"].str.contains("АТМ")
substring_2 = df_tr_types["tr_description"].str.contains("POS")
match_substring = substring_1 | substring_2
types_after_match = df_tr_types["tr_type"][match_substring]
count_true = df_tr_types.isin(types_after_match).value_counts()[True]
print(count_true / count_tr_types)

tr_description
False    0.062
dtype: float64


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [5]:
occurrence_of_types = df_transactions["tr_type"].value_counts()

top_types_from_occerrence = df_tr_types[df_tr_types["tr_type"].isin(occurrence_of_types[0:10].index)]
top_types_from_occerrence

,tr_type,tr_description
39,7010,Взнос наличных через АТМ (в своем тер.банке)
47,7030,Перевод на карту (с карты) через АТМ (в предел...
59,7070,Перевод на карту (с карты) через Мобильный бан...
60,7071,Перевод на карту (с карты) через Мобильный бан...
98,1010,Покупка. POS ТУ СБ РФ
99,1030,Оплата услуги. Банкоматы СБ РФ
100,1100,Покупка. ТУ Россия
101,1110,Покупка. POS ТУ Россия
106,2010,Выдача наличных в АТМ Сбербанк России
129,2370,Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [6]:
positive_transactions = df_transactions[df_transactions["amount"] > 0]
group_customers_by_amount = positive_transactions.groupby("customer_id").sum()
sorted_table_clients = group_customers_by_amount.sort_values("amount", ascending=False)
client_with_max_added_sum = sorted_table_clients.iloc[0]
client_with_max_added_sum

mcc_code    5.307003e+08
tr_type     6.236914e+08
amount      1.248115e+09
Name: 70780820, dtype: float64

In [10]:
negative_transactions = df_transactions[df_transactions["amount"] < 0]
group_customers_by_amount_2 = negative_transactions.groupby("customer_id").sum()
sorted_table_clients_2 = group_customers_by_amount_2.sort_values("amount", ascending=True)
client_with_max_subtracted_sum = sorted_table_clients_2.iloc[0]
client_with_max_subtracted_sum

mcc_code    2.686385e+06
tr_type     1.093690e+06
amount     -1.249952e+09
Name: 70780820, dtype: float64

In [11]:
abs(client_with_max_added_sum["amount"] - abs(client_with_max_subtracted_sum["amount"]))

1837317.980000019

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [12]:
top_transactions_from_occerrence_types = df_transactions[df_transactions["tr_type"].isin(occurrence_of_types[:10].index)]

mean = top_transactions_from_occerrence_types.groupby("tr_type")["amount"].mean()
median = top_transactions_from_occerrence_types.groupby("tr_type")["amount"].median()
pd.concat([mean, median], keys=["mean", "median"], axis=1, levels=True)

,mean,median
tr_type,,
1010,-19784.748640,-7411.52
1030,-5320.980222,-2245.92
1100,-44061.827262,-10188.26
1110,-32119.330371,-11207.57
2010,-136077.629325,-44918.32
2370,-205418.249032,-44918.32
7010,276391.789596,112295.79
7030,86104.332909,13951.52
7070,65569.831700,11319.42


In [13]:
first_id = sorted_table_clients.idxmax()["amount"]
second_id = sorted_table_clients_2.idxmax()["amount"]
# first_id == second_id равны в нашем случае, 
# но алгоритм написан на случай, если данные в Data будут изменены

transactions_ = df_transactions.loc[df_transactions["customer_id"] == (first_id or second_id)]

mean = transactions_.groupby("tr_type")["amount"].mean()
median = transactions_.groupby("tr_type")["amount"].median()
pd.concat([mean, median], keys=["mean", "median"], axis=1, levels=True)

,mean,median
tr_type,,
2010,-2.941056e+06,-3368873.660
2330,-2.382398e+06,-2245915.770
2370,-2.218505e+06,-2245915.770
6110,1.756293e+04,10028.010
7020,1.465129e+04,10319.985
7021,2.850067e+04,28500.670
7030,1.406196e+04,8848.910
7031,1.921036e+04,12992.620
7034,1.430648e+04,14306.480


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [14]:
df_transactions = pd.merge(df_transactions, df_gender_train, how='left')
df_transactions = pd.merge(df_transactions, df_tr_mcc_codes, how='inner')
df_transactions = pd.merge(df_transactions, df_tr_types, how='inner')
df_transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [15]:
negative_trasactions = df_transactions[df_transactions["amount"] < 0].groupby("gender")["amount"].mean()
abs(negative_trasactions[0] - negative_trasactions[1])


32718.054920224655

In [16]:
positive_trasactions = df_transactions[df_transactions["amount"] > 0].groupby("gender")["amount"].mean()
abs(positive_trasactions[0] - positive_trasactions[1])


63366.57104801464

## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [17]:
max_income = df_transactions[df_transactions["amount"] > 0].groupby(["gender", "tr_type"])["amount"].sum()
max_income_top_10_selection_0_gender =  max_income[0].sort_values()[:10]
max_income_top_10_selection_0_gender

tr_type
4051     1122.96
4210     2245.92
4110     2245.92
2370     2335.75
7075     6737.75
2110    22459.16
1010    31532.66
4100    40334.88
2210    68315.82
1210    83644.87
Name: amount, dtype: float64

In [18]:
max_income_top_10_selection_1_gender =  max_income[1].sort_values()[:10]
max_income_top_10_selection_1_gender

tr_type
2020      2245.92
4110      4491.84
1010     35317.03
8100     64682.37
6000     79280.83
7041     87590.72
2370    113194.15
7015    121279.45
2110    179673.26
2010    224591.59
Name: amount, dtype: float64

In [19]:
max_income_top_10_selection_1_gender.to_frame().merge(max_income_top_10_selection_0_gender.to_frame(), how='inner', on='tr_type')

,amount_x,amount_y
tr_type,,
4110,4491.84,2245.92
1010,35317.03,31532.66
2370,113194.15,2335.75
2110,179673.26,22459.16


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [20]:
mcc_costs = df_transactions[df_transactions["amount"] < 0].groupby(["gender", "mcc_code"])["amount"].sum()
mcc_costs

gender  mcc_code
0.0     742          -386207.68
        1711         -674897.69
        1799         -261626.73
        2741          -20746.42
        3000       -74431645.27
                       ...     
1.0     8999       -28643346.63
        9222         -988205.17
        9311         -373271.22
        9399        -1810820.67
        9402           -4581.67
Name: amount, Length: 349, dtype: float64

In [21]:
top_modulo_expenses = abs(mcc_costs[0] - mcc_costs[1]).sort_values(ascending=False)[:10]
df_tr_mcc_codes[df_tr_mcc_codes["mcc_code"].isin(top_modulo_expenses.index)]

,mcc_code,mcc_description
22,4829,Денежные переводы
62,5511,"Легковой и грузовой транспорт — продажа, серви..."
64,5533,Автозапчасти и аксессуары
65,5541,Станции техобслуживания
75,5661,Обувные магазины
90,5812,"Места общественного питания, рестораны"
116,5977,Магазины косметики
123,6010,Финансовые институты — снятие наличности вручную
124,6011,Финансовые институты — снятие наличности автом...
162,7995,Транзакции по азартным играм


## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

не делал